In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.10_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.10_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "32.22494345718902,674.6843840085317,564.1769951534734,158.11386192027007"
                              ],
                          columns=[
                              "46.17305331179322,175.0728271405493,213.55037156704364,252.027915993538,290.50546042003236,320.3255573505655,360.72697899838454,400.16646203554126,441.52982229402267,473.2737964458805,522.3326655896608"
                              ],
                          split_text=True,
                          strip_text='\n\t',
                          row_tol=15,
                          pages="all"
                         )

In [6]:
tables

<TableList n=1>

In [7]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [8]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
printnAllTables(tables)

Page No:  1
    0                                             1       2       3      4   \
0                                         Rural Area                          
1    1  Near School/College/ Educational institution   18753   14609   4206   
2    2                         Near Residential Area   75201   58760  16045   
3    3                          Near Religious Place   13191   10766   2846   
4    4            Near Recreation Place/ Cinema Hall   11715    9824   2578   
5    5                                  Near Factory   12937   10682   2391   
6    6                                        Others  128582  118443  22788   
7    7                             Sub Total (Rural)  260379  223084  50854   
8                                         Urban Area                          
9    8  Near School/College/ Educational institution   18390   14414   3821   
10   9                         Near Residential Area   55742   41126  10430   
11  10                          Near Rel

In [9]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [10]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [11]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)   

In [12]:
def set_Column(df):
    column_list = [
        ("Sl. No.",''),
        ("Cause",''), 
        ("No. of Cases",''),
        ('No. of Persons Injured','Male'),
        ('No. of Persons Injured','Female'),
        ('No. of Persons Injured','Transgender'),
        ('No. of Persons Injured','Total'),
        ('No. of Persons Died','Male'),
        ('No. of Persons Died','Female'),
        ('No. of Persons Died','Transgender'),
        ('No. of Persons Died','Total'),
        ('Percentage Share (Cases)','')
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [13]:
for i in range(len(tables)):
    set_Column(tables[i].df)

In [14]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [15]:
# Reset Index before saving file, for better formatting in RAW CSV
tables[0].df.to_csv("Place of Occurrence - wise Road Accident Cases, Persons Injured and Died during 2019 (All India).csv",index=False)